In [ ]:
!pip install amplify

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.9 MB 4.3 MB/s 


In [ ]:
import numpy as np

# 都市数
N = 5

# 都市間の移動距離
Q = np.array([[1000, 20, 20, 50, 40],
              [30, 1000, 10, 30, 20],
              [20, 10, 1000, 30, 20],
              [50, 30, 20, 1000, 10],
              [40, 20, 20, 10, 1000]])

In [ ]:
# 変数を作成
from amplify import BinarySymbolGenerator
gen = BinarySymbolGenerator()
x = gen.array(N, N)
x

[[ q_0,  q_1,  q_2,  q_3,  q_4],
 [ q_5,  q_6,  q_7,  q_8,  q_9],
 [q_10, q_11, q_12, q_13, q_14],
 [q_15, q_16, q_17, q_18, q_19],
 [q_20, q_21, q_22, q_23, q_24]]

In [ ]:
# x.roll(-1, axis=0)は時刻t+1の時の変数s
x.roll(-1, axis=0)

[[ q_5,  q_6,  q_7,  q_8,  q_9],
 [q_10, q_11, q_12, q_13, q_14],
 [q_15, q_16, q_17, q_18, q_19],
 [q_20, q_21, q_22, q_23, q_24],
 [ q_0,  q_1,  q_2,  q_3,  q_4]]

In [ ]:
# コスト関数
from amplify import einsum
cost = einsum("ij,ni,nj->", Q, x, x.roll(-1, axis=0))
cost

1000 q_0 q_5 + 20 q_0 q_6 + 20 q_0 q_7 + 50 q_0 q_8 + 40 q_0 q_9 + 1000 q_0 q_20 + 30 q_0 q_21 + 20 q_0 q_22 + 50 q_0 q_23 + 40 q_0 q_24 + 30 q_1 q_5 + 1000 q_1 q_6 + 10 q_1 q_7 + 30 q_1 q_8 + 20 q_1 q_9 + 20 q_1 q_20 + 1000 q_1 q_21 + 10 q_1 q_22 + 30 q_1 q_23 + 20 q_1 q_24 + 20 q_2 q_5 + 10 q_2 q_6 + 1000 q_2 q_7 + 30 q_2 q_8 + 20 q_2 q_9 + 20 q_2 q_20 + 10 q_2 q_21 + 1000 q_2 q_22 + 20 q_2 q_23 + 20 q_2 q_24 + 50 q_3 q_5 + 30 q_3 q_6 + 20 q_3 q_7 + 1000 q_3 q_8 + 10 q_3 q_9 + 50 q_3 q_20 + 30 q_3 q_21 + 30 q_3 q_22 + 1000 q_3 q_23 + 10 q_3 q_24 + 40 q_4 q_5 + 20 q_4 q_6 + 20 q_4 q_7 + 10 q_4 q_8 + 1000 q_4 q_9 + 40 q_4 q_20 + 20 q_4 q_21 + 20 q_4 q_22 + 10 q_4 q_23 + 1000 q_4 q_24 + 1000 q_5 q_10 + 20 q_5 q_11 + 20 q_5 q_12 + 50 q_5 q_13 + 40 q_5 q_14 + 30 q_6 q_10 + 1000 q_6 q_11 + 10 q_6 q_12 + 30 q_6 q_13 + 20 q_6 q_14 + 20 q_7 q_10 + 10 q_7 q_11 + 1000 q_7 q_12 + 30 q_7 q_13 + 20 q_7 q_14 + 50 q_8 q_10 + 30 q_8 q_11 + 20 q_8 q_12 + 1000 q_8 q_13 + 10 q_8 q_14 + 40 q_9 q_10 + 20 

In [ ]:
from amplify.constraint import one_hot

# 制約条件1：各都市に1回は訪問しなければならない
constr_1 = [one_hot(x[n]) for n in range(N)]

# 制約条件2：1度に訪れる都市は1つでなければならない
constr_2 = [one_hot(x[:, i]) for i in range(N)]

constraints = sum(constr_1) + sum(constr_2)

In [ ]:
# 制約条件1：各都市に1回は訪問しなければならない
constr_1

[q_0 + q_1 + q_2 + q_3 + q_4 == 1,
 q_5 + q_6 + q_7 + q_8 + q_9 == 1,
 q_10 + q_11 + q_12 + q_13 + q_14 == 1,
 q_15 + q_16 + q_17 + q_18 + q_19 == 1,
 q_20 + q_21 + q_22 + q_23 + q_24 == 1]

In [ ]:
# 制約条件2：1度に訪れる都市は1つでなければならない
constr_2

[q_0 + q_5 + q_10 + q_15 + q_20 == 1,
 q_1 + q_6 + q_11 + q_16 + q_21 == 1,
 q_2 + q_7 + q_12 + q_17 + q_22 == 1,
 q_3 + q_8 + q_13 + q_18 + q_23 == 1,
 q_4 + q_9 + q_14 + q_19 + q_24 == 1]

In [ ]:
lam = 100 # penalty係数
model = cost + lam*constraints

In [ ]:
# クライアントの設定
from amplify.client import FixstarsClient
client = FixstarsClient()
client.parameters.timeout = 1000  # タイムアウト1秒
client.parameters.outputs.duplicate = True # 同じエネルギー値で合っても異なる解であれば考慮する
client.token = "Amplifyのtokenを入力してください"

In [ ]:
# 組み合わせ最適化を実行
from amplify import Solver
solver = Solver(client)
result = solver.solve(model)

if len(result) == 0:
  raise RuntimeError("Any one of constraints is not satisfied.")
else:
  print("Number of found answers:", len(result))

energy, values = result[0].energy, result[0].values

Number of found answers: 5


In [ ]:
# 結果を表示
for i in result:
  print(i.energy, i.values)

90.0 {2: 0, 23: 0, 1: 0, 17: 0, 22: 1, 10: 0, 14: 1, 20: 0, 11: 0, 24: 0, 8: 0, 19: 0, 21: 0, 15: 0, 9: 0, 7: 0, 6: 1, 5: 0, 16: 0, 13: 0, 3: 0, 4: 0, 18: 1, 12: 0, 0: 1}
90.0 {2: 0, 23: 0, 1: 0, 17: 0, 22: 0, 10: 0, 14: 0, 20: 0, 11: 0, 24: 0, 8: 1, 19: 0, 21: 1, 15: 1, 9: 0, 7: 0, 6: 0, 5: 0, 16: 0, 13: 0, 3: 0, 4: 1, 18: 0, 12: 1, 0: 0}
90.0 {2: 0, 23: 0, 1: 0, 17: 0, 22: 0, 10: 1, 14: 0, 20: 0, 11: 0, 24: 1, 8: 0, 19: 0, 21: 0, 15: 0, 9: 0, 7: 1, 6: 0, 5: 0, 16: 1, 13: 0, 3: 1, 4: 0, 18: 0, 12: 0, 0: 0}
90.0 {2: 0, 23: 0, 1: 1, 17: 1, 22: 0, 10: 0, 14: 0, 20: 1, 11: 0, 24: 0, 8: 0, 19: 0, 21: 0, 15: 0, 9: 1, 7: 0, 6: 0, 5: 0, 16: 0, 13: 1, 3: 0, 4: 0, 18: 0, 12: 0, 0: 0}
90.0 {2: 1, 23: 1, 1: 0, 17: 0, 22: 0, 10: 0, 14: 0, 20: 0, 11: 1, 24: 0, 8: 0, 19: 1, 21: 0, 15: 0, 9: 0, 7: 0, 6: 0, 5: 1, 16: 0, 13: 0, 3: 0, 4: 0, 18: 0, 12: 0, 0: 0}


In [ ]:
# 結果を整形して表示
for i in result:
  print("energy:", i.energy)
  print("result:")
  print(x.decode(i.values), "\n")

energy: 90.0
result:
[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]] 

energy: 90.0
result:
[[0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]] 

energy: 90.0
result:
[[0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]] 

energy: 90.0
result:
[[0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]] 

energy: 90.0
result:
[[0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]] 



In [ ]:
# 制約条件を守っているかを確認
model.check_constraints(result[0].values)

[((q_0 + q_1 + q_2 + q_3 + q_4 == 1, 1000), True),
 ((q_5 + q_6 + q_7 + q_8 + q_9 == 1, 1000), True),
 ((q_10 + q_11 + q_12 + q_13 + q_14 == 1, 1000), True),
 ((q_15 + q_16 + q_17 + q_18 + q_19 == 1, 1000), True),
 ((q_20 + q_21 + q_22 + q_23 + q_24 == 1, 1000), True),
 ((q_0 + q_5 + q_10 + q_15 + q_20 == 1, 1000), True),
 ((q_1 + q_6 + q_11 + q_16 + q_21 == 1, 1000), True),
 ((q_2 + q_7 + q_12 + q_17 + q_22 == 1, 1000), True),
 ((q_3 + q_8 + q_13 + q_18 + q_23 == 1, 1000), True),
 ((q_4 + q_9 + q_14 + q_19 + q_24 == 1, 1000), True)]